In [ ]:
import tensorflow as tf
import matplotlib as plt
import numpy as np

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
def display_digit(num, x, y, vector = None):
    label = y[num]
    image = x[num]
    if vector is None:
        plt.title('Example: {}  Label: {}'.format(num, label))
        plt.imshow(image, cmap=plt.get_cmap('gray_r'))
    else:
        plt.figure(figsize=(6,3))
        plt.subplot(1,2,1)
        plt.title('Real label: {}'.format(label))
        plt.imshow(image, cmap=plt.get_cmap('gray_r'))
        plt.subplot(1,2,2)
        thisplot = plt.bar(range(10), vector, color="#777777")
        plt.ylim([0, 1]) 
        plt.xticks([])
        plt.yticks([])
        predicted_label = np.argmax(vector)
        thisplot[predicted_label].set_color('red')
        plt.title('Predicted label: {}'.format(predicted_label))
    plt.show()

In [ ]:
model1 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(100, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])
history1 = model1.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

In [ ]:
model2 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(100, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])
history2 = model2.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))